# Tests



In [1]:
# import needed libraries
import numpy as np
from sklearn.decomposition import PCA
import sys
import os

# Get the current working directory (make sure to adjust the path accordingly)
current_dir = os.path.dirname(os.path.abspath("__file__"))

# Add the path of the decomposition_lib to the system path
sys.path.append(os.path.join(current_dir, 'plots'))
import plots
sys.path.append(os.path.join(current_dir, 'emg_lib'))
import emg_lib as emgl
import matplotlib.pyplot as plt


# Add the path of the decomposition_lib to the system path
sys.path.append(os.path.join(current_dir, 'decomposition_lib'))
import decomposition_lib as dl
import scipy.io 
from scipy.signal import butter, filtfilt, iirnotch



Figure closed!


## Extend

In [3]:
fs = 2048
exfactor = 16
nbIterations = 20
#emg_filt = np.array([[1, 2], [3, 4]])
# Load the MATLAB file
mat_file  = scipy.io.loadmat('sample_data\hdemg_SOL_sample.mat')
# Load the variable, e.g., if it's named 'emg_filt' in MATLAB
# Assuming it might be structured; convert to numpy array
if 'EMG' in mat_file:
    emg = np.array(mat_file['EMG'])
    emg = emg.T
    #emg = emg[:,fs:-fs]
else:
    raise ValueError("Variable 'emg_filt' not found in the MATLAB file.")

emg_filt, emg_mask, emg_envelope = emgl.filter_emg(emg,fs)

emg_filt = emg_filt[:,4*fs:-4*fs]

# plt.figure
# plt.plot(emg_filt[0,:])
# plt.show()
# plt.pause(0.1)


MUFilters, SIL, COV, PulseT, Distime, normIPT, centroids = \
    dl.decomposition_offline(emg_filt, fs, exfactor,nbIterations, preOptFilters=None, refineStrategy='SIL', \
                              showPlots=True, h=None, ax=None, peeloff_flag=False, removeDuplicates=True, qc_threshold=0.85)

Removing duplicates:   0%|          | 0/20 [00:00<?, ?it/s]


TypeError: correlate() got multiple values for argument 'mode'

In [ ]:
# for debugging:

In [ ]:
# Ensure interactive mode is set up for Jupyter Notebook
from plots.live_plots import LivePlot

# Initialize the live plot with the desired configuration
live_plot = LivePlot(num_axes=2, num_lines=[1, 2])

# Simulated data update in a loop (example for demonstration)
import numpy as np
import time

x_data = np.linspace(0, 10, 100)
for i in range(100):
    y_data1 = np.sin(x_data + i * 0.1)
    y_data2 = np.cos(x_data + i * 0.1)
    
    # Update with new data
    live_plot.update([
        [(x_data, y_data1)],  # For the first axis
        [(x_data, y_data1), (x_data, y_data2)]  # For the second axis
    ])
    
    time.sleep(0.1)  # Simulate real-time data feed

In [ ]:
import matplotlib.pyplot as plt
plt.figure
plt.plot(filtered_emg[0,:])
plt.show()
plt.pause(0.1)